<mark> **Please make a copy of this notebook before running it** </mark>

#Introduction: What is Cardinal?

Cardinal is a Python library, created by the [Open Contracting Partnership](https://www.open-contracting.org/) to measure red flags and procurement indicators using OCDS data.

Currently, Cardinal supports the calculation of the following red flags:

* The percentage difference between the winning bid and the second-lowest valid bid is a low outlier [(R024)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/024.html)
* The ratio of winning bids to submitted bids for a top tenderer is a low outlier [(R025)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/025.html)
* Bids are disqualified if not submitted by the single tenderer of the winning bid [(R035)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/035.html)
* The lowest submitted bid is disqualified, while the award criterion is price only [(R036)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/036.html)
* The ratio of disqualified bids to submitted bids is a high outlier per buyer, procuring entity or tenderer [(R038)](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/038.html)


To read the full documentation you can visit the [documentation site](https://cardinal.readthedocs.io/en/latest/).

In this tutorial we explain how to run Cardinal in a Google Colaboratory notebook.  We will cover the following steps:

1. Setup
2. Collecting data from the [OCDS Data Registry](https://data.open-contracting.org/)
3. Preparting the data
4. Exploring the data
5. Calculating the indicators


If you have any questions running this notebook you can write to **data@open-contracting.org**

# Setup

Install [OCDS Cardinal](https://github.com/open-contracting/cardinal-rs/blob/main/README.md) and other libraries.

In [1]:
! curl -sSOL https://github.com/open-contracting/cardinal-rs/releases/download/0.0.5/ocdscardinal-0.0.5-linux-64-bit.zip

In [2]:
! unzip -oj ocdscardinal-0.0.5-linux-64-bit.zip ocdscardinal-0.0.5-linux-64-bit/ocdscardinal
! ls

Archive:  ocdscardinal-0.0.5-linux-64-bit.zip
  inflating: ocdscardinal            
ocdscardinal  ocdscardinal-0.0.5-linux-64-bit.zip  sample_data


In [3]:
import pandas as pd
from google.colab import data_table
%load_ext google.colab.data_table

# Download the data from the Data Registry

To select the data source go to the [OCDS data registry](https://data.open-contracting.org/) and select the desired publisher.  For the publisher of choice select a **jsonl file** and copy the url, in the command below.

**In the registry you will also find a description of the data source and direct links to the publisher website where you can find more information about the scope of the publication.**

<img src="https://drive.google.com/uc?id=10dlm8c55pN89YTGEyZgvsLDc8fFMLNf0"  width="200" height="300">

In this example we will use data from the Dominican Republic.  

In [14]:
url = input('Add URL from the registry source:')

Add URL from the registry source:https://data.open-contracting.org/api/download_export?spider=dominican_republic_api&job_id=1289&full=true&suffix=jsonl.gz


In [15]:
! curl -sSOJ "$url"

In the files tab at the left hand side of the notebook, look for the file ending in .gz you just downloaded (e.g `chile_compra_api_releases_full.jsonl.gz`), and add it to the command below (see example):

<img src="https://drive.google.com/uc?id=19z86Nj5OY7Y8REfcd2sZbFPXDTAWZYS6" width="200" height="200">


In [6]:
file=input('Add name of .gz file: ')

Add name of .gz file: dominican_republic_api_full.jsonl.gz


In [7]:
file_jsonl=file.replace('.gz', '')

In [ ]:
! gunzip -f $file

In [9]:
! ls -lh $file_jsonl

ls: cannot access 'dominican_republic_api_full.jsonl': No such file or directory


# Prepare the data

###See which fields are published

Before you start calculating any indicators you can use the [`coverage`](https://cardinal.readthedocs.io/en/latest/cli/coverage.html) command, to see which fields are published in the dataset.

In [10]:
! ./ocdscardinal coverage  $file_jsonl >> result_fields.json

error: dominican_republic_api_full.jsonl: No such file or directory (os error 2)

Usage: ocdscardinal [OPTIONS] <COMMAND>

For more information, try '--help'.


In [12]:
fields = pd.DataFrame(pd.read_json('result_fields.json', typ='series'), columns=['releases']).rename_axis('path').reset_index()
#Leaves only object members
fields=fields[fields.path.str.contains('[a-z]$')]
fields['path']=fields['path'].str.replace(r'[][]|^/', '', regex=True)
fields_list=fields['path'].tolist()
fields

ValueError: Expected object or value

### Write a default settings file

Use the [`init`](https://cardinal.readthedocs.io/en/latest/cli/init.html) command to write a default settings file for configuration.

In [ ]:
! ./ocdscardinal init settings.ini

Settings written to "settings.ini".


Preview the content of the settings file. You can edit the [settings file](https://cardinal.readthedocs.io/en/latest/topics/settings.html) as needed, to configure the behavior of the `prepare` and `indicators` commands.  As a default, the settings file enables all indicators.

In [ ]:
! ./ocdscardinal init -

; currency = USD

; `prepare` command
;
; Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/prepare.html

[defaults]
; currency = USD
; item_classification_scheme = UNSPSC
; bid_status = valid
; award_status = active

[codelists.BidStatus]
; qualified = valid

[codelists.AwardStatus]
; Active = active

; `indicators` command
;
; Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/indicators/

[R024]
; threshold = 0.05

[R025]
; percentile = 75
; threshold = 0.05

[R035]
; threshold = 1

[R036]

[R038]
; threshold = 0.5


### Identify and review data quality issues

You can use the [`prepare`](https://cardinal.readthedocs.io/en/latest/cli/prepare.html) command to identify if the dataset has data quality issues and correct them before calculating the indicators.

In [ ]:
! ./ocdscardinal prepare --settings settings.ini --output prepared.jsonl --errors issues.csv $file_jsonl

Check the issues reported:

In [ ]:
! wc issues.csv

  2659605   5122634 250305685 issues.csv


In [ ]:
! head -n 10 issues.csv

1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,0,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,1,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,2,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,3,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,4,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,5,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,6,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,7,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,8,Qualified,invalid
1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,9,Qualified,invalid


Read issues into a data frame:

In [ ]:
import pandas as pd

df = pd.read_csv("issues.csv", header=None)
df.columns = ["line", "ocid", "path", "array indexes", "incorrect value", "error description"]
df.head()

,line,ocid,path,array indexes,incorrect value,error description
0,1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,0.0,Qualified,invalid
1,1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,1.0,Qualified,invalid
2,1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,2.0,Qualified,invalid
3,1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,3.0,Qualified,invalid
4,1,ocds-6550wx-LMD-DAF-CM-2021-0006,/bids/details[]/status,4.0,Qualified,invalid


Check which errors occur per path.  In this case the dataset does not have a classification scheme set and has invalid values in the bids status and some entries without the value set.  

In [ ]:
df[["path", "error description"]].value_counts().reset_index()

,path,error description,0
0,/awards[]/items[]/classification/scheme,not set,1820107
1,/bids/details[]/status,invalid,830667
2,/bids/details[]/status,not set,8831


Check the invalid values in the bids status:

In [ ]:
df[(df['error description']=='invalid')].groupby('incorrect value').size().reset_index()

,incorrect value,0
0,Disqualified,117749
1,InTreatment,33773
2,Qualified,679145


You can correct the data quality issues in the settings file.  You can [read in the documentation](https://cardinal.readthedocs.io/en/latest/cli/prepare.html#configuration) how to:


* Normalize ID fields
* Fill in missing values
* Re-map incorrect codes

For example, we can set the value in the settings file:


```
[defaults]
item_classification_scheme = UNSPSC
```


Once you re-write the settings file with the new configurations, you can re-run the prepare command.  In this example we will set the award status.

In [ ]:
%%writefile settings.ini
[defaults]
item_classification_scheme = UNSPSC

[codelists.BidStatus]
Qualified = valid
Disqualified = disqualified
InTreatment = pending

[codelists.AwardStatus]
; Active = active

; `indicators` command
;
; Read the documentation at:
; https://cardinal.readthedocs.io/en/latest/cli/indicators/

[R024]
; threshold = 0.05

[R025]
; percentile = 75
; threshold = 0.05

[R035]
; threshold = 1

[R036]

[R038]
minimum_submitted_bids = 2

Overwriting settings.ini


Re-run the prepare command with the new settings.

In [ ]:
! ./ocdscardinal prepare --settings settings.ini --output prepared.jsonl --errors issues.csv $file_jsonl

# Calculate indicators

You can calculate the indicators using the [`indicators`](https://cardinal.readthedocs.io/en/latest/cli/indicators/index.html) command.  The command will calculate all the indicators enabled in the settings file.

The command below writes the output in `results.json`

In [ ]:
! ./ocdscardinal indicators -s settings.ini prepared.jsonl > results.json

Check the number of results:

In [ ]:
! wc -c results.json

1660400 results.json


### Review the output

You can read in detail in [the documentation](https://cardinal.readthedocs.io/en/latest/cli/indicators/index.html#demonstration) how to interpret the output.

The JSON output is organized as an object in which the key is a group: “OCID” (the unique identifier of a contracting process), “Buyer”, “ProcuringEntity” or “Tenderer”.


```
{
  "OCID": {},
  "Buyer": {},
  "ProcuringEntity": {},
  "Tenderer": {}
}
```
Each value at the top level is an object representing the results within that group, in which the key is an identifier extracted from the input data.  Each value at the second level is an object representing the results relating to that identifier, in which the key is the code for an indicator, and the value is the output of that indicator.

```
{
  "OCID": {
    "ocds-6550wx-JRFPFA-DAF-CM-2021-0012": {
      "R036": 1.0
    }
  },
  "Buyer": {
    "DO-RPE-55216": {
      "R038": 0.8
    }
  }
}
```
The JSON output also has a Meta key at the top level. Its value is an object with information about the quartiles and fences used to calculate the results, rounded to 4 decimals.

```
{
  // ...
  "Meta": {
    "R024": {
      "q1": 66.6667,
      "q3": 100.0,
      "lower_fence": 16.6667
    }
  }
}
```

The output below shows the values of the configurations of the indicators in the Meta key.

In [ ]:
import json
from pprint import pprint
with open("results.json") as f:
    data = json.load(f)
data['Meta']

{'R038': {'ProcuringEntity_q1': 0.0282,
  'ProcuringEntity_q3': 0.2149,
  'ProcuringEntity_upper_fence': 0.4949,
  'Tenderer_q1': 0.0,
  'Tenderer_q3': 0.2063,
  'Tenderer_upper_fence': 0.5159},
 'R025': {'q1': 0.5714,
  'q3': 1.0,
  'upper_fence': 11.0,
  'lower_fence': -0.0714},
 'R024': {'q1': 0.0285, 'q3': 0.2757, 'lower_fence': -0.3422}}

In the data variable the results are stored in a dictionary.  Since the flags can be calculated by contracting process (ocid), tenderer or procuring entity, we can explore the results for each.  

In [ ]:
data

{'Tenderer': {'DO-RPE-97590': {'R038': 0.75},
  'DO-RPE-72574': {'R038': 1.0},
  'DO-RPE-8830': {'R038': 0.75},
  'DO-RPE-88823': {'R038': 1.0},
  'DO-RPE-21195': {'R038': 0.6666666666666666},
  'DO-RPE-102965': {'R038': 0.6666666666666666},
  'DO-RPE-23251': {'R038': 0.5526315789473685},
  'DO-RPE-70841': {'R038': 0.6666666666666666},
  'DO-RPE-39896': {'R038': 0.7142857142857143},
  'DO-RPE-83955': {'R038': 0.6666666666666666},
  'DO-RPE-66200': {'R038': 0.6666666666666666},
  'DO-RPE-43380': {'R038': 0.5714285714285714},
  'DO-RPE-88521': {'R038': 0.6363636363636364},
  'DO-RPE-90679': {'R038': 1.0},
  'DO-RPE-5354': {'R038': 0.6153846153846154},
  'DO-RPE-35427': {'R038': 0.6666666666666666},
  'DO-RPE-108574': {'R038': 0.75},
  'DO-RPE-5439': {'R038': 1.0},
  'DO-RPE-39273': {'R038': 0.6},
  'DO-RPE-11130': {'R038': 1.0},
  'DO-RPE-94947': {'R038': 0.6666666666666666},
  'DO-RPE-21223': {'R038': 0.6},
  'DO-RPE-47497': {'R038': 1.0},
  'DO-RPE-10736': {'R038': 0.8},
  'DO-RPE-1011

####Explore flags by ocid.

The flags [R036](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/036.html) (lowest bid disqualified) and [R035](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/035.html) (All except winning bid disqualified) were calculated.  

In [ ]:
ocid = pd.DataFrame.from_dict(data["OCID"], orient="index").reset_index()
ocid.head()

,index,R036,R035
0,ocds-6550wx-ARD-DAF-CM-2021-0006,1.0,3.0
1,ocds-6550wx-DGAP-UC-CD-2018-0848,1.0,NaN
2,ocds-6550wx-CODOPESCA-DAF-CM-2021-0008,1.0,NaN
3,ocds-6550wx-CONAVIHSIDA-CCC-CP-2020-0001,1.0,1.0
4,ocds-6550wx-INDOTEL-CCC-CP-2022-0014,1.0,NaN


29121 procedures were flagged for disqualifying the lowest bid when the award criteria was price only.  

In [ ]:
len(ocid)

29121

#### Explore flags by tenderer

The flag [R038](https://cardinal.readthedocs.io/en/latest/cli/indicators/R/038.html) (Excessive disqualified bids) was calculated.

In [ ]:
tenderer = pd.DataFrame.from_dict(data["Tenderer"], orient="index").reset_index()
tenderer.head()

,index,R038
0,DO-RPE-100020,1.000000
1,DO-RPE-100065,0.666667
2,DO-RPE-100283,0.666667
3,DO-RPE-100445,1.000000
4,DO-RPE-100446,1.000000


Count the number of flags per tenderer

In [ ]:
tenderer_flags=pd.melt(tenderer, id_vars=['index'], value_vars=['R038'])

tenderer_flags[~tenderer_flags.value.isna()].groupby('index')\
.agg(total_flags=('variable', 'count')).sort_values(by='total_flags', ascending=False).reset_index()

,index,total_flags
0,DO-RPE-100020,1
1,DO-RPE-76258,1
2,DO-RPE-76532,1
3,DO-RPE-77088,1
4,DO-RPE-77207,1
...,...,...
912,DO-RPE-36084,1
913,DO-RPE-36224,1
914,DO-RPE-36345,1
915,DO-RPE-36371,1


There are 292 tenderers with all their bids disqualified.  

In [ ]:
import plotly.express as px
fig = px.histogram(tenderer, x="R038", marginal = 'box', template='plotly_white',
                   title="R038 distribution tenderers")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=600, height=400)
fig.show()

In [ ]:
#Filter tenderers with all their bids diqualified
R038_tenderer=tenderer[tenderer['R038']==1]

You can explore the results to see if they make sense.  To explore the data we will use [flatterer](https://flatterer.opendata.coop/) to flatten the json into csv tables.  

In [ ]:
!pip install flatterer

In [ ]:
import flatterer
output = flatterer.flatten('dominican_republic_api_full.jsonl', 'dom', json_stream=True)

Most of the bidders with all their bids disqualified had submitted 2 bids.  You can decide to adjust the threshold

In [ ]:
#Load the bid tenderers table.
bids_tenderers=pd.read_csv('/content/dom/csv/bids_details_tenderers.csv')

#Filter only flagged bidders with a 100% of bids disqualified, and calculate the number of bids.
bidders_distribution=bids_tenderers[bids_tenderers['id'].isin(R038_tenderer['index'])]\
.groupby('id').agg(total=('id', 'count')).reset_index()

#Plot the distribution.
fig = px.histogram(bidders_distribution, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()

#### Explore flags by procuring entity

In [ ]:
procuring_entity = pd.DataFrame.from_dict(data["ProcuringEntity"], orient="index").reset_index()
procuring_entity.head()

,index,R038
0,1010,0.517241
1,1033,0.666667
2,1039,0.666667
3,1047,0.647059
4,1081,0.583333


In [ ]:
len(procuring_entity)

25

In [ ]:
fig = px.histogram(procuring_entity, x="R038", marginal = 'box', template='plotly_white',
                   title="R038 distribution procuring entity")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=600, height=400)
fig.show()

Check the number of procedures of the flagged entities.

In [ ]:
#Load the tenders (main) table.
tenders=pd.read_csv('/content/dom/csv/main.csv')

In [ ]:
entities_distribution=tenders[tenders['tender_procuringEntity_id'].astype(str).isin(procuring_entity['index'])]\
.groupby('tender_procuringEntity_id').agg(total=('tender_procuringEntity_id', 'count')).reset_index()

fig = px.histogram(entities_distribution, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()

The entities flagged have very few procedures, so we can check the distribution of the number of procedures by entity for the full dataset, to see if we should adjust the threshold.  

The results show that Q1=6, so we could use this threshold `minimum_contracting_processes=6` in the settings file and re-run the indicators.

In [ ]:
entities_distribution1=tenders.groupby('tender_procuringEntity_id')\
.agg(total=('tender_procuringEntity_id', 'count')).reset_index()

fig = px.histogram(entities_distribution1, x="total", marginal = 'box', template='plotly_white',
                   title="Number of bids by flagged bidders (R038)")
fig.update_traces(marker_color='#D6E100')
fig.update_layout(width=500, height=400)
fig.show()